2. Configuración del Entorno Colab y Carga de Datos (Celdas 1 y 2)

2.1 Instalación de Librerías

In [1]:
# CELDA 1: Instalaciones
!pip install stable-baselines3 gym pandas scikit-learn numpy
import gym
from gym import spaces
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from stable_baselines3 import PPO # Algoritmo de Aprendizaje por Refuerzo
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 2.4 MB/s  0:00:00m ? eta -:--:--
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827728 sha256=fcd04b7b12fe34a2a63e2e04eba1b0ea6624a943d491ace62d789b53c11ecbcd
  Stored in directory: /home/will/.cache/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gym]


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


2.2 Carga y Preprocesamiento de Datos:

El archivo debe ser una serie de tiempo con datos a intervalos regulares (idealmente horarios), abarcando al menos un año.

In [2]:
# CELDA 2: Carga de Datos
from google.colab import files
# Sube tu archivo 'datos_microrred_sc_p.csv' (con las columnas de demanda, solar, eolica, hora, etc.)
# files.upload()

# ⚠️ Reemplazar 'datos_microrred_sc_p.csv' con tu archivo real
df = pd.read_csv('datos_microrred_sc_p.csv')

# Asegurar que el DataFrame esté ordenado por tiempo
df['Tiempo'] = pd.to_datetime(df['Tiempo']) # Asume una columna de tiempo
df = df.sort_values('Tiempo').reset_index(drop=True)

# Crear features de tiempo si no existen
if 'Hora' not in df.columns:
    df['Hora'] = df['Tiempo'].dt.hour
    df['Dia_Semana'] = df['Tiempo'].dt.dayofweek

print("Datos cargados y listos. Filas totales:", len(df))

ModuleNotFoundError: No module named 'google.colab'

3. Implementación del Random Forest (RF) (Celda 3)
El RF se entrena para predecir la Demanda, la Generación Solar y la Generación Eólica de la siguiente hora ($\mathbf{t+1}$), basándose en las condiciones actuales ($\mathbf{t}$).

In [ ]:
# CELDA 3: Entrenamiento del Random Forest

# Definir Features (X) y Targets (Y)
features = ['Hora', 'Dia_Semana', 'Temperatura', 'Radiacion_Solar', 'Velocidad_Viento'] # Asegúrate que estas columnas existan
X = df[features]

# Los Targets son los valores de la siguiente hora (shift(-1))
y_demanda = df['Demanda'].shift(-1).fillna(df['Demanda'].mean())
y_solar = df['Solar_Gen'].shift(-1).fillna(df['Solar_Gen'].mean())
y_eolica = df['Eolica_Gen'].shift(-1).fillna(df['Eolica_Gen'].mean())

# Entrenar los tres modelos RF
rf_demanda = RandomForestRegressor(n_estimators=100, random_state=42)
rf_solar = RandomForestRegressor(n_estimators=100, random_state=42)
rf_eolica = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar con todos los datos menos el último paso (que no tiene t+1)
rf_demanda.fit(X.iloc[:-1], y_demanda.iloc[:-1])
rf_solar.fit(X.iloc[:-1], y_solar.iloc[:-1])
rf_eolica.fit(X.iloc[:-1], y_eolica.iloc[:-1])

print("Modelos Random Forest entrenados y listos para la predicción híbrida.")

NameError: name 'df' is not defined

4. Definición del Entorno de Microrred (RL-Random Forest) (Celda 4)
Se crea el ambiente de Gym (MicrogridEnv) que simula la física de la microrred y utiliza las predicciones del RF.

Ajusta los Parámetros Físicos y Económicos de la Isla en la función __init__.

In [ ]:
# =========================================================================
# 1. PARÁMETROS CLAVE DEL MODELO (Basados en la ZNI Providencia y Santa Catalina)
# =========================================================================

# Parámetros para el Entorno MicrogridEnv (Celda 4)
DIESEL_P_MAX = 3500.0   # Capacidad Instalada (kW) - Estimación basada en escala operativa de EEDAS.
BATT_CAP_MAX = 1500.0   # Capacidad Máxima de Batería (kWh) - Basado en proyectos de almacenamiento.
DIESEL_COST_KWH = 0.55  # Costo de Generación (USD/kWh) - Alto costo del diésel en ZNI.
SOLAR_CAP_MW = 1.0      # Capacidad instalada solar (1000 kWp) - Basado en proyectos de FNCE.
EÓLICA_CAP_KW = 500.0   # Capacidad instalada eólica (kW).

# Parámetros para la Generación de Demanda (Celda 2)
AÑOS_SIMULACION = 3
DIESEL_P_PICO = 2800.0  # Pico de Demanda Máxima (kW) - 80% de DIESEL_P_MAX para margen.
PERIODO_TOTAL = AÑOS_SIMULACION * 8760 # Horas totales

# Perfil de Carga Normalizado (Basado en la imagen de la curva de carga)
PERFIL_NORMALIZADO = [
    0.55, 0.40, 0.35, 0.30, 0.25, 0.32, 0.45, 0.60, 0.75, 0.80, 0.75, 0.70,
    0.65, 0.60, 0.55, 0.50, 0.55, 0.65, 0.75, 0.85, 0.95, 1.00, 0.90, 0.70
]

# Factores de Ajuste Estacional (Simulando Turismo: Ene, Jul, Dic son más altos)
FACTORES_ESTACIONALES = {
    1: 1.05, 2: 1.02, 3: 1.00, 4: 0.98, 5: 0.95, 6: 1.00,
    7: 1.05, 8: 1.02, 9: 0.98, 10: 0.95, 11: 1.00, 12: 1.05
}
# =========================================================================

import numpy as np
import pandas as pd

# --- 2. CREACIÓN DEL DATAFRAME BASE ---
tiempo = pd.date_range(start='2021-01-01 00:00:00', periods=PERIODO_TOTAL, freq='H')
df = pd.DataFrame(index=tiempo)
df['Hora'] = df.index.hour
df['Dia_Semana'] = df.index.dayofweek
df['Mes'] = df.index.month

# --- 3. GENERACIÓN DE LA DEMANDA SINTÉTICA (TARGET) ---
# 1. Aplicar Perfil Básico y Escala:
df['Demanda_Base'] = df['Hora'].apply(lambda h: PERFIL_NORMALIZADO[h] * DIESEL_P_PICO)

# 2. Aplicar Ajuste Estacional:
df['Factor_Estacional'] = df['Mes'].map(FACTORES_ESTACIONALES)
df['Demanda_Ajustada'] = df['Demanda_Base'] * df['Factor_Estacional']

# 3. Aplicar Ruido Aleatorio (Variabilidad Diaria ± 3%):
ruido = np.random.uniform(low=-0.03, high=0.03, size=PERIODO_TOTAL)
df['Demanda'] = df['Demanda_Ajustada'] * (1 + ruido)
df['Demanda'] = df['Demanda'].clip(lower=0) # Asegurar valores positivos

# --- 4. GENERACIÓN DE FEATURES CLIMÁTICOS Y RENOVABLES ---

# Perfil de Radiación Solar (Simulación de 1000 W/m2 de pico, con ruido)
df['Radiacion_Solar'] = 1000 * np.sin(np.pi * (df['Hora'] - 6) / 12).clip(lower=0)
df['Radiacion_Solar'] += np.random.normal(0, 50, PERIODO_TOTAL) * (df['Radiacion_Solar'] > 0)
df['Radiacion_Solar'] = df['Radiacion_Solar'].clip(lower=0)

# Generación Solar (Usando la capacidad instalada SOLAR_CAP_MW = 1000 kWp)
df['Solar_Gen'] = SOLAR_CAP_MW * 1000.0 * (df['Radiacion_Solar'] / 1000.0) * 0.8
df['Solar_Gen'] = df['Solar_Gen'].clip(upper=SOLAR_CAP_MW * 1000.0, lower=0)

# Perfil de Velocidad del Viento (Tendencia ligeramente mayor en la noche/madrugada)
v_viento_base = 8.0 + 3.0 * np.cos(2 * np.pi * (df['Hora'] - 4) / 24)
df['Velocidad_Viento'] = v_viento_base + np.random.normal(0, 1.5, PERIODO_TOTAL)
df['Velocidad_Viento'] = df['Velocidad_Viento'].clip(lower=0)

# Generación Eólica (Usando la capacidad instalada EÓLICA_CAP_KW = 500 kW)
# Se usa una curva de potencia cúbica simplificada (P = 0 si V < 3 m/s)
df['Eolica_Gen'] = np.where(df['Velocidad_Viento'] > 3,
                            EÓLICA_CAP_KW * (df['Velocidad_Viento'] / 12)**3, 0)
df['Eolica_Gen'] = df['Eolica_Gen'].clip(upper=EÓLICA_CAP_KW)

# Temperatura (Correlacionada con la Demanda y con ruido)
df['Temperatura'] = 28.0 + 3.0 * np.sin(np.pi * (df['Hora'] - 10) / 12).clip(lower=-1)
df['Temperatura'] += np.random.normal(0, 1.0, PERIODO_TOTAL)
df['Temperatura'] = df['Temperatura'].clip(lower=24.0)


# --- 5. RESULTADO FINAL (DataFrame listo para el modelo) ---
df_final = df[['Demanda', 'Solar_Gen', 'Eolica_Gen', 'Hora', 'Dia_Semana', 'Temperatura', 'Radiacion_Solar', 'Velocidad_Viento']]

print(f"DataFrame 'df_final' generado con {len(df_final)} horas de datos sintéticos (3 años).")
print("\nPrimeras 5 filas del DataFrame:")
print(df_final.head())

DataFrame 'df_final' generado con 26280 horas de datos sintéticos (3 años).

Primeras 5 filas del DataFrame:
                         Demanda  Solar_Gen  Eolica_Gen  Hora  Dia_Semana  \
2021-01-01 00:00:00  1641.201332        0.0  181.293781     0           4   
2021-01-01 01:00:00  1166.390771        0.0  266.420794     1           4   
2021-01-01 02:00:00  1032.419576        0.0  193.130981     2           4   
2021-01-01 03:00:00   859.263252        0.0  239.943406     3           4   
2021-01-01 04:00:00   752.780000        0.0  145.969400     4           4   

                     Temperatura  Radiacion_Solar  Velocidad_Viento  
2021-01-01 00:00:00    25.885521              0.0          8.556948  
2021-01-01 01:00:00    25.157696              0.0          9.728531  
2021-01-01 02:00:00    24.612897              0.0          8.739271  
2021-01-01 03:00:00    24.000000              0.0          9.394944  
2021-01-01 04:00:00    27.008581              0.0          7.960589  


/tmp/ipython-input-734818618.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  tiempo = pd.date_range(start='2021-01-01 00:00:00', periods=PERIODO_TOTAL, freq='H')
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


5. Entrenamiento del Agente RL (PPO) (Celda 5)

Se utiliza el algoritmo de optimización de política PPO (Proximal Policy Optimization), ideal para problemas de control continuo.

In [ ]:
# CELDA 5: Entrenamiento del Agente RL

# Inicializar y entrenar el modelo PPO
# NOTA: Aumenta 'total_timesteps' para mejor rendimiento (ej. 500,000 a 1,000,000)
model = PPO("MlpPolicy", env, verbose=1, gamma=env.DISCOUNT_RATE, learning_rate=3e-4)

print("Comenzando el entrenamiento del Agente PPO (Esto tomará tiempo)...")
model.learn(total_timesteps=100000)
print("Entrenamiento completado.")

NameError: name 'env' is not defined

6. Simulación y Comparación de Escenarios (Celda 6)

Para demostrar el valor de la transición, debes comparar el despacho aprendido (RL) contra un escenario base de solo diésel.

6.1 Simulación del Escenario Híbrido Óptimo (RL)

In [ ]:
# CELDA 6: Simulación y Evaluación
def run_simulation(env, model=None, is_rl=True):
    obs, info = env.reset()
    done = False
    total_cost = 0
    log = []

    while not done:
        idx = env.current_step
        P_load = env.df.iloc[idx]['Demanda']
        P_solar = env.df.iloc[idx]['Solar_Gen']
        P_eolica = env.df.iloc[idx]['Eolica_Gen']

        if is_rl and model is not None:
            # Despacho RL-Random Forest
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _, info = env.step(action)
            costo_paso = -reward
            P_diesel_usado = env.soc * env.DIESEL_COST_KWH # Simplificado para log

        else:
            # Escenario Línea Base (Solo Diésel, ignorando renovables/batería para el costo)
            P_req = P_load
            P_diesel_usado = P_req
            costo_paso = P_diesel_usado * env.DIESEL_COST_KWH

            # Ajustar el SOC para que la simulación de costos sea justa
            env.soc = env.BATT_CAP_MAX / 2.0 # Mantener SOC inactivo

            # Simulación de paso
            env.current_step += 1
            done = env.current_step >= len(env.df) - 1
            obs = env._get_obs() # Actualizar estado para el siguiente paso (si es necesario)


        total_cost += costo_paso
        log.append({
            'Tiempo': env.df.iloc[idx]['Tiempo'],
            'Demanda': P_load,
            'Solar_Gen': P_solar,
            'Eolica_Gen': P_eolica,
            'P_Diesel_Usado': P_diesel_usado,
            'Costo': costo_paso,
            'SOC': env.soc if is_rl else env.BATT_CAP_MAX / 2.0 # SOC solo relevante para RL
        })

    return pd.DataFrame(log), total_cost

# Ejecutar las dos simulaciones
log_rl, cost_rl = run_simulation(env, model, is_rl=True)
log_base, cost_base = run_simulation(env, model=None, is_rl=False)

print("\n--- Resultados de la Simulación ---")
print(f"Costo Total Base (Solo Diésel): ${cost_base:,.2f}")
print(f"Costo Total Óptimo (RL-Híbrido): ${cost_rl:,.2f}")
print(f"Ahorro de Costo: ${cost_base - cost_rl:,.2f}")

NameError: name 'env' is not defined

7. Análisis de Resultados y Visualización (Celda 7)

Este paso genera los gráficos necesarios para el informe.

In [ ]:
# CELDA 7: Visualización

# 1. Comparación de Despacho Diésel (RL vs. Base)
plt.figure(figsize=(12, 6))
plt.plot(log_base['Tiempo'], log_base['P_Diesel_Usado'], label='Diésel (Base)', color='red', linestyle='--')
plt.plot(log_rl['Tiempo'], log_rl['P_Diesel_Usado'], label='Diésel (RL-Híbrido)', color='blue')
plt.title('Comparación de Uso de Generación Diésel')
plt.xlabel('Tiempo')
plt.ylabel('Potencia Diésel (kW)')
plt.legend()
plt.grid(True)
plt.show()

# 2. Visualización del SOC de la Batería (Solo RL)
plt.figure(figsize=(12, 6))
plt.plot(log_rl['Tiempo'], log_rl['SOC'], label='Estado de Carga (SOC)', color='green')
plt.title('Comportamiento del Estado de Carga (SOC) con Despacho RL')
plt.xlabel('Tiempo')
plt.ylabel('SOC (kWh)')
plt.legend()
plt.grid(True)
plt.show()

# 3. Comparación de costos totales
ahorro_porcentaje = (cost_base - cost_rl) / cost_base * 100
print(f"\nEl modelo RL-Híbrido logra una reducción del {ahorro_porcentaje:.2f}% en costos operativos.")

NameError: name 'log_base' is not defined

<Figure size 1200x600 with 0 Axes>